In [ ]:
# pixel_to_coords.py
# Author: Shreya Singh Rathour
# Date: 09-07-2024
# Version: 1.1
# Description: This script efficiently converts pixel locations in an image (including GIFs) to coordinates
#              in various coordinate systems, visualizes the results, and
#              provides an option to save and download the visualized image in PNG format.

# Import required libraries
import numpy as np  # For efficient numerical operations
from google.colab import files  # For file handling in Google Colab
from IPython.display import display  # For displaying images in the notebook
import io  # For handling byte streams
from multiprocessing import Pool  # For parallel processing
from functools import lru_cache  # For caching function results
from PIL import Image, ImageDraw, ImageFont  # For image processing and drawing

@lru_cache(maxsize=None)  # Decorator to cache function results
def convert_pixel_to_coords(image_size, pixel_location, coord_system='top-left'):
    """
    Convert a single pixel location to coordinates (cached for efficiency).
    """
    width, height = image_size  # Unpack image dimensions
    px, py = pixel_location  # Unpack pixel coordinates
    if px < 0 or px >= width or py < 0 or py >= height:  # Check if pixel is within image bounds
        raise ValueError(f"Pixel coordinates {pixel_location} are outside the image bounds")
    if coord_system == 'top-left':  # Convert based on top-left coordinate system
        return px, py
    elif coord_system == 'center':  # Convert based on center coordinate system
        return px - width // 2, height // 2 - py
    elif coord_system == 'bottom-left':  # Convert based on bottom-left coordinate system
        return px, height - py
    else:  # Raise error for invalid coordinate system
        raise ValueError(f"Invalid coordinate system: {coord_system}")

def convert_pixels_to_coords_np(image_size, pixel_locations, coord_system='top-left'):
    """
    Convert multiple pixel locations to coordinates using NumPy for efficiency.
    """
    width, height = image_size  # Unpack image dimensions
    pixels = np.array(pixel_locations)  # Convert pixel locations to NumPy array

    # Check if any pixel is out of bounds
    if np.any((pixels[:, 0] < 0) | (pixels[:, 0] >= width) | (pixels[:, 1] < 0) | (pixels[:, 1] >= height)):
        raise ValueError("Some pixel coordinates are outside the image bounds")

    if coord_system == 'top-left':  # Return original pixels for top-left system
        return pixels
    elif coord_system == 'center':  # Convert to center coordinate system
        return np.column_stack((pixels[:, 0] - width // 2, height // 2 - pixels[:, 1]))
    elif coord_system == 'bottom-left':  # Convert to bottom-left coordinate system
        return np.column_stack((pixels[:, 0], height - pixels[:, 1]))
    else:  # Raise error for invalid coordinate system
        raise ValueError(f"Invalid coordinate system: {coord_system}")

def process_chunk(args):
    """
    Process a chunk of pixel locations for parallel processing.
    """
    chunk, image_size, coord_system = args  # Unpack arguments
    return convert_pixels_to_coords_np(image_size, chunk, coord_system)  # Convert chunk of pixels

def parallel_convert(image_size, pixel_locations, coord_system, num_processes=4):
    """
    Convert pixel locations to coordinates using parallel processing.
    """
    chunk_size = max(1, len(pixel_locations) // num_processes)  # Calculate chunk size
    chunks = [pixel_locations[i:i + chunk_size] for i in range(0, len(pixel_locations), chunk_size)]  # Create chunks
    with Pool(num_processes) as p:  # Create a pool of processes
        return np.concatenate(p.map(process_chunk, [(chunk, image_size, coord_system) for chunk in chunks]))  # Process chunks in parallel

def visualize_conversions(image, pixel_locations, converted_coords):
    """
    Visualize pixel to coordinate conversions on the image using PIL.
    """
    draw = ImageDraw.Draw(image)  # Create a drawing object
    font = ImageFont.load_default()  # Load default font
    for (px, py), (x, y) in zip(pixel_locations, converted_coords):  # Iterate over pixel locations and converted coordinates
        draw.ellipse([px-3, py-3, px+3, py+3], fill=(0, 255, 0))  # Draw green circle at pixel location
        draw.text((px + 5, py - 5), f"({x}, {y})", fill=(0, 255, 0), font=font)  # Draw converted coordinates
    return image  # Return the modified image

def main():
    """
    Main function to handle image upload, pixel input, and conversion visualization.
    """
    print("Please upload an image file (including GIFs).")
    uploaded = files.upload()  # Upload file

    if not uploaded:  # Check if file was uploaded
        print("No file was uploaded.")
        return

    filename = list(uploaded.keys())[0]  # Get filename
    file_content = uploaded[filename]  # Get file content

    # Open image with PIL to support various formats including GIF
    img = Image.open(io.BytesIO(file_content))

    if img is None:  # Check if image was successfully opened
        print(f"Error: Unable to read the image {filename}")
        return

    # For GIFs, use only the first frame
    if img.format == 'GIF':
        img = img.convert('RGB')  # Convert GIF to RGB
        print("Note: For GIF images, only the first frame is processed.")

    coord_system = input("Enter coordinate system (top-left/center/bottom-left): ").lower()  # Get coordinate system from user
    if coord_system not in ['top-left', 'center', 'bottom-left']:  # Validate coordinate system
        print("Invalid coordinate system. Using 'top-left' as default.")
        coord_system = 'top-left'

    pixel_locations = []  # Initialize list to store pixel locations
    while True:  # Loop to get pixel locations from user
        try:
            pixel_x = int(input("Enter the X coordinate of the pixel: "))  # Get X coordinate
            pixel_y = int(input("Enter the Y coordinate of the pixel: "))  # Get Y coordinate
            pixel_locations.append((pixel_x, pixel_y))  # Add pixel location to list
            if input("Enter another pixel location? (yes/no): ").lower() != 'yes':  # Check if user wants to enter more pixels
                break
        except ValueError:
            print("Invalid input. Please enter integers for X and Y coordinates.")

    try:
        image_size = img.size  # Get image size
        converted_coords = parallel_convert(image_size, pixel_locations, coord_system)  # Convert pixel locations to coordinates

        for (px, py), (x, y) in zip(pixel_locations, converted_coords):  # Print conversion results
            print(f"Pixel location ({px}, {py}) corresponds to coordinates: x={x}, y={y}")

        visualized_img = visualize_conversions(img, pixel_locations, converted_coords)  # Visualize conversions on image

        # Display the image
        display(visualized_img)

        if input("Save and download the visualized image? (yes/no): ").lower() == 'yes':  # Check if user wants to save image
            output_filename = f'visualized_{filename.rsplit(".", 1)[0]}.png'  # Generate output filename
            visualized_img.save(output_filename)  # Save image
            files.download(output_filename)  # Download image
            print(f"Image saved and downloaded as {output_filename}")
    except ValueError as e:  # Handle ValueError
        print(f"Error: {e}")
    except Exception as e:  # Handle other exceptions
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()  # Run main function if script is executed directly

In [ ]:
# pixel_to_coords.py
# Author: Shreya Singh Rathour (
# Date: 09-07-2024
# Version: 1.2
# Update: In this update we efficiently converts pixel locations in an image (including GIFs) to coordinates
#              in various coordinate systems, visualizes the results, and
#              provides an option to save and download the visualized image in its original format.

# Import required libraries
import numpy as np  # For efficient numerical operations
from google.colab import files  # For file handling in Google Colab environment
from IPython.display import display  # For displaying images in Jupyter notebooks
import io  # For handling byte streams
from multiprocessing import Pool  # For parallel processing
from functools import lru_cache  # For caching function results
from PIL import Image, ImageDraw, ImageFont  # For image processing and drawing

@lru_cache(maxsize=None)  # Cache all results of this function
def convert_pixel_to_coords(image_size, pixel_location, coord_system='top-left'):
    """
    Convert a single pixel location to coordinates (cached for efficiency).
    """
    width, height = image_size  # Unpack image dimensions
    px, py = pixel_location  # Unpack pixel coordinates

    # Check if pixel is within image bounds
    if px < 0 or px >= width or py < 0 or py >= height:
        raise ValueError(f"Pixel coordinates {pixel_location} are outside the image bounds")

    # Convert based on the specified coordinate system
    if coord_system == 'top-left':
        return px, py  # No conversion needed for top-left system
    elif coord_system == 'center':
        return px - width // 2, height // 2 - py  # Convert to center-based coordinates
    elif coord_system == 'bottom-left':
        return px, height - py  # Convert to bottom-left based coordinates
    else:
        raise ValueError(f"Invalid coordinate system: {coord_system}")

def convert_pixels_to_coords_np(image_size, pixel_locations, coord_system='top-left'):
    """
    Convert multiple pixel locations to coordinates using NumPy for efficiency.
    """
    width, height = image_size  # Unpack image dimensions
    pixels = np.array(pixel_locations)  # Convert pixel locations to NumPy array

    # Check if any pixel is out of bounds using vectorized operations
    if np.any((pixels[:, 0] < 0) | (pixels[:, 0] >= width) | (pixels[:, 1] < 0) | (pixels[:, 1] >= height)):
        raise ValueError("Some pixel coordinates are outside the image bounds")

    # Convert based on the specified coordinate system
    if coord_system == 'top-left':
        return pixels  # No conversion needed for top-left system
    elif coord_system == 'center':
        return np.column_stack((pixels[:, 0] - width // 2, height // 2 - pixels[:, 1]))  # Convert to center-based coordinates
    elif coord_system == 'bottom-left':
        return np.column_stack((pixels[:, 0], height - pixels[:, 1]))  # Convert to bottom-left based coordinates
    else:
        raise ValueError(f"Invalid coordinate system: {coord_system}")

def process_chunk(args):
    """
    Process a chunk of pixel locations for parallel processing.
    """
    chunk, image_size, coord_system = args  # Unpack arguments
    return convert_pixels_to_coords_np(image_size, chunk, coord_system)  # Convert chunk of pixels

def parallel_convert(image_size, pixel_locations, coord_system, num_processes=4):
    """
    Convert pixel locations to coordinates using parallel processing.
    """
    chunk_size = max(1, len(pixel_locations) // num_processes)  # Calculate chunk size
    chunks = [pixel_locations[i:i + chunk_size] for i in range(0, len(pixel_locations), chunk_size)]  # Create chunks
    with Pool(num_processes) as p:  # Create a pool of processes
        return np.concatenate(p.map(process_chunk, [(chunk, image_size, coord_system) for chunk in chunks]))  # Process chunks in parallel and concatenate results

def visualize_conversions(image, pixel_locations, converted_coords):
    """
    Visualize pixel to coordinate conversions on the image using PIL.
    """
    draw = ImageDraw.Draw(image)  # Create a drawing object
    font = ImageFont.load_default()  # Load default font
    for (px, py), (x, y) in zip(pixel_locations, converted_coords):  # Iterate over pixel locations and converted coordinates
        draw.ellipse([px-3, py-3, px+3, py+3], fill=(0, 255, 0))  # Draw green circle at pixel location
        draw.text((px + 5, py - 5), f"({x}, {y})", fill=(0, 255, 0), font=font)  # Draw converted coordinates
    return image  # Return the modified image

def main():
    """
    Main function to handle image upload, pixel input, and conversion visualization.
    """
    print("Please upload an image file (including GIFs).")
    uploaded = files.upload()  # Upload file using Google Colab's file uploader

    if not uploaded:  # Check if a file was uploaded
        print("No file was uploaded.")
        return

    filename = list(uploaded.keys())[0]  # Get the filename of the uploaded file
    file_content = uploaded[filename]  # Get the content of the uploaded file

    # Open image with PIL to support various formats including GIF
    img = Image.open(io.BytesIO(file_content))

    if img is None:  # Check if the image was successfully opened
        print(f"Error: Unable to read the image {filename}")
        return

    # For GIFs, use only the first frame
    if img.format == 'GIF':
        img = img.convert('RGB')  # Convert GIF to RGB format
        print("Note: For GIF images, only the first frame is processed.")

    coord_system = input("Enter coordinate system (top-left/center/bottom-left): ").lower()  # Get coordinate system from user
    if coord_system not in ['top-left', 'center', 'bottom-left']:  # Validate coordinate system
        print("Invalid coordinate system. Using 'top-left' as default.")
        coord_system = 'top-left'

    pixel_locations = []  # Initialize list to store pixel locations
    while True:  # Loop to get pixel locations from user
        try:
            pixel_x = int(input("Enter the X coordinate of the pixel: "))  # Get X coordinate
            pixel_y = int(input("Enter the Y coordinate of the pixel: "))  # Get Y coordinate
            pixel_locations.append((pixel_x, pixel_y))  # Add pixel location to list
            if input("Enter another pixel location? (yes/no): ").lower() != 'yes':  # Check if user wants to enter more pixels
                break
        except ValueError:
            print("Invalid input. Please enter integers for X and Y coordinates.")

    try:
        image_size = img.size  # Get image dimensions
        converted_coords = parallel_convert(image_size, pixel_locations, coord_system)  # Convert pixel locations to coordinates

        for (px, py), (x, y) in zip(pixel_locations, converted_coords):  # Print conversion results
            print(f"Pixel location ({px}, {py}) corresponds to coordinates: x={x}, y={y}")

        visualized_img = visualize_conversions(img, pixel_locations, converted_coords)  # Visualize conversions on image

        # Display the image
        display(visualized_img)

        if input("Save and download the visualized image? (yes/no): ").lower() == 'yes':  # Check if user wants to save image
            output_filename = 'visualized_' + filename  # Generate output filename
            visualized_img.save(output_filename)  # Save image in the original format
            files.download(output_filename)  # Download the saved image
            print(f"Image saved and downloaded as {output_filename}")

    except ValueError as e:  # Handle ValueError
        print(f"Error: {e}")
    except Exception as e:  # Handle other exceptions
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    main()  # Run the main function if the script is executed directly